code copy and filter data to eliminate knife class

In [5]:
import os
import glob
from multiprocessing import Pool, cpu_count



# Function to process a single file
def process_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    new_lines = []
    for line in lines:
        class_id = line.split()[0]
        if class_id == '0':
            continue  # Remove class 0
        elif class_id == '1':
            new_line = line.replace('1', '0', 1)
            new_lines.append(new_line)
        else:
            new_lines.append(line)

    with open(file_path, 'w') as file:
        file.writelines(new_lines)

# Get all label files
label_directory = '/mnt/storage/kilsar_jainil/archive-3/archive-3/dataset/dataset/labels/val/'
label_files = glob.glob(os.path.join(label_directory, '*.txt'))
print(len(label_files))

label_directory = '/mnt/storage/kilsar_jainil/archive-3/archive-3/dataset/dataset/labels/train/'
label_files_2 = glob.glob(os.path.join(label_directory, '*.txt'))
print(len(label_files_2))
# Limit number of cores to 32
num_cores = min(60, cpu_count())

with Pool(num_cores) as pool:
    pool.map(process_file, label_files)
    pool.map(process_file, label_files_2)

print("Processing complete.")



156
4000


Processing complete.


In [1]:
# Fix the directory structure by copying it to mohammad's folder

In [6]:
import os
import shutil
from multiprocessing import Pool, cpu_count

def copy_file(args):
    src, dest = args
    if not os.path.exists(os.path.dirname(dest)):
        os.makedirs(os.path.dirname(dest), exist_ok=True)
    shutil.copy2(src, dest)

def get_all_files(src_folder):
    file_paths = []
    for root, _, files in os.walk(src_folder):
        for file in files:
            full_path = os.path.join(root, file)
            file_paths.append(full_path)
    return file_paths

def restructure_and_copy(src_folder, dest_folder):
    all_files = get_all_files(src_folder)
    
    # Map source files to their new destination paths
    tasks = []
    for file in all_files:
        relative_path = os.path.relpath(file, src_folder)
        new_path = os.path.join(dest_folder, os.path.dirname(relative_path), os.path.basename(file))
        tasks.append((file, new_path))
    
    # Use multiprocessing to copy files
    with Pool(cpu_count()) as pool:
        pool.map(copy_file, tasks)

if __name__ == "__main__":
    src_folder = '/mnt/storage/kilsar_jainil/archive-3/archive-3/dataset/dataset/labels/train/'
    dest_folder = '/mnt/storage/kilsar_mohammad/archive-3/train/labels/'
    restructure_and_copy(src_folder, dest_folder)
    
    src_folder = '/mnt/storage/kilsar_jainil/archive-3/archive-3/dataset/dataset/images/train/'
    dest_folder = '/mnt/storage/kilsar_mohammad/archive-3/train/images/'
    restructure_and_copy(src_folder, dest_folder)
    
    src_folder = '/mnt/storage/kilsar_jainil/archive-3/archive-3/dataset/dataset/labels/val/'
    dest_folder = '/mnt/storage/kilsar_mohammad/archive-3/val/labels/'
    restructure_and_copy(src_folder, dest_folder)
    
    src_folder = '/mnt/storage/kilsar_jainil/archive-3/archive-3/dataset/dataset/images/val/'
    dest_folder = '/mnt/storage/kilsar_mohammad/archive-3/val/images/'
    restructure_and_copy(src_folder, dest_folder)


In [7]:
import os
import shutil
from multiprocessing import Pool, cpu_count

def find_and_remove_empty_label(args):
    label_path, image_extension = args
    with open(label_path, 'r') as label_file:
        content = label_file.read().strip()
        if not content:
            # The label file is empty; remove the corresponding image
            image_path = label_path.replace('/labels/', '/images/').replace('.txt', image_extension)
            if os.path.exists(image_path):
                os.remove(image_path)
                print(f'Removed image: {image_path}')
            os.remove(label_path)  # Optionally, remove the empty label file
            print(f'Removed label: {label_path}')

def get_all_label_files(label_folder):
    label_files = []
    for root, _, files in os.walk(label_folder):
        for file in files:
            if file.endswith('.txt'):
                full_path = os.path.join(root, file)
                label_files.append(full_path)
    return label_files

def remove_images_with_empty_labels(label_folder, image_extension='.jpg'):
    all_label_files = get_all_label_files(label_folder)
    tasks = [(label_file, image_extension) for label_file in all_label_files]
    
    # Use multiprocessing to process the label files
    with Pool(cpu_count()) as pool:
        pool.map(find_and_remove_empty_label, tasks)

if __name__ == "__main__":
    label_folder = '/mnt/storage/kilsar_mohammad/archive-3/train/labels/'
    image_extension = '.jpg'
    remove_images_with_empty_labels(label_folder, image_extension)
    
    label_folder = '/mnt/storage/kilsar_mohammad/archive-3/val/labels/'
    image_extension = '.jpg'
    remove_images_with_empty_labels(label_folder, image_extension)



Removed image: /mnt/storage/kilsar_mohammad/archive-3/train/images/3223.jpgRemoved image: /mnt/storage/kilsar_mohammad/archive-3/train/images/2935.jpg

Removed image: /mnt/storage/kilsar_mohammad/archive-3/train/images/3552.jpg
Removed image: /mnt/storage/kilsar_mohammad/archive-3/train/images/2121.jpgRemoved label: /mnt/storage/kilsar_mohammad/archive-3/train/labels/3223.txt
Removed image: /mnt/storage/kilsar_mohammad/archive-3/train/images/3010.jpg
Removed label: /mnt/storage/kilsar_mohammad/archive-3/train/labels/2935.txt

Removed label: /mnt/storage/kilsar_mohammad/archive-3/train/labels/3552.txtRemoved image: /mnt/storage/kilsar_mohammad/archive-3/train/images/2369.jpgRemoved image: /mnt/storage/kilsar_mohammad/archive-3/train/images/2591.jpg


Removed image: /mnt/storage/kilsar_mohammad/archive-3/train/images/3876.jpgRemoved label: /mnt/storage/kilsar_mohammad/archive-3/train/labels/2121.txtRemoved image: /mnt/storage/kilsar_mohammad/archive-3/train/images/2706.jpg

Removed image